In [1]:
#Knapsack Problem

In [2]:
import genetic
import datetime
import numpy as np
import copy
from functools import reduce
import random
import time
from math import sqrt, exp

In [3]:
class GenFuncs:
    def __init__(self, genes, individual, initialize, fitness, mutate, sample, vis=None):
        self._genes = genes
        self._individual = individual
        self._initialize = initialize
        self._fitness = fitness
        self._mutate = mutate
        self._sample = sample
        self._vis = vis
        
    def initialize(self, lambda_):
        return self._initialize(lambda_, self._genes, self._individual, self._fitness)
    
    def mutate(self, ind, gen):
        return self._mutate(ind, gen, self._fitness, self._individual)
    
    def sample(self, population, mu, gen):
        return self._sample(population, mu, gen, self._fitness)
    
    def visualize(self, ind, start_time):
        return self._vis(ind, start_time)
            


In [4]:
#Define the genes

class Resource:
    def __init__(self, name, value, weight, volume):
        self.name = name
        self.value = value
        self.weight = weight
        self.volume = volume

genes = {Resource('Flour', 1680, .265, .41):0,
         Resource('Butter', 1440, .5, .13):0,
         Resource('Sugar', 1840, .441, .29):0}

global max_weight, max_volume
max_weight = 10
max_volume = 4



In [5]:
#Define the individual
        
class Knapsack(genetic.Individual):    
    def __init__(self, genome, fitness, age=0):
        self.genome = genome
        self.fitness = fitness
        self.age = 0
        self.available_weight = max_weight - fitness.total_weight
        self.available_volume = max_volume - fitness.total_volume

In [6]:
#Define the initialization

def initialize(lambda_, genes, individual, fitness):
    init_pop = []
    for n in range(lambda_):
        cand = copy.copy(genes)
        fit = fitness(genes)
        while True:
            new_genes = copy.copy(cand)
            new_genes[random.choice(list(new_genes.keys()))] += 1
            new_fit = fitness(new_genes)
            if new_fit.total_weight > max_weight or new_fit.total_volume > max_volume:
                break
            else:
                cand = new_genes
                fit = new_fit
        init_pop.append(individual(cand, fit))
    return init_pop

In [7]:
#Define the fitness

class Fitness:
    def __init__(self, genes):
        self.total_weight = sum([x.weight*genes[x] for x in list(genes.keys())])
        self.total_volume = sum([x.volume*genes[x] for x in list(genes.keys())])
        self.total_value = sum([x.value*genes[x] for x in list(genes.keys())])
        
    def __ne__(self, other):
        if self.total_value == other.total_value and \
        self.total_weight == other.total_weight and \
        self.total_volume == other.total_volume:
            return True
        else:
            return False
                       
    def __gt__(self, other):
        if self.total_value != other.total_value:
            return self.total_value > other.total_value
        elif self.total_weight != other.total_weight:
            return self.total_weight < other.total_weight
        elif self.total_volume != other.total_volume:
            return self.total_volume < other.total_volume
        else:
            return False
    
    def __lt__(self, other):
        if other == np.inf:
            return True


def get_fitness(genes):
    return Fitness(genes)

In [8]:
#define mutation

def add_item(genes):
    new_genes = copy.copy(genes)
    target = random.choice(list(new_genes.keys()))
    new_genes[target] += 1
    return new_genes

def del_item(genes):
    new_genes = copy.copy(genes)
    target = random.choice(list(new_genes.keys()))
    while new_genes[target] == 0:
        target = random.choice(list(new_genes.keys()))
    new_genes[target] -= 1
    return new_genes

def mutate(ind, gen, fitness, ind_class):
    genome = add_item(ind.genome)
    fit = fitness(genome)
    while fit.total_weight > max_weight or fit.total_volume > max_volume:
        genome = del_item(genome)
        fit = fitness(genome)
    return ind_class(genome, fit)

In [9]:
#Define sampling

def truncation_selection(population, n, gen, fitness, anneal=False):
    population.sort(key=lambda x: x.fitness, reverse=True)
    if anneal is True:
        individual = population[n-1]
        test = population[n]
        if exp(-(individual-population[n])/(1/log(gen))) > random.random():
            population[n-1] = test
            population[n] = individual
    return population[:n]

In [10]:
#Define display

def display(ind, start_time):
    print(time.time()-start_time)
    dic = {x.name:ind.genome[x] for x in list(ind.genome.keys())}
    print(f'The individual\'s genome: \n {dic} \n and fitness: {ind.fitness.total_value}')

In [11]:
knapsack = GenFuncs(genes, Knapsack, initialize, get_fitness,
            mutate, truncation_selection, display)

In [14]:
knap_es = genetic.ES(knapsack, 1, 1, timer=5)  

In [16]:
a = knap_es.solve()

0.0001728534698486328
The individual's genome: 
 {'Flour': 5, 'Butter': 5, 'Sugar': 4} 
 and fitness: 22960
0.00028824806213378906
The individual's genome: 
 {'Flour': 5, 'Butter': 6, 'Sugar': 4} 
 and fitness: 24400
0.0015549659729003906
The individual's genome: 
 {'Flour': 3, 'Butter': 6, 'Sugar': 6} 
 and fitness: 24720
0.0016410350799560547
The individual's genome: 
 {'Flour': 3, 'Butter': 7, 'Sugar': 6} 
 and fitness: 26160
0.0016942024230957031
The individual's genome: 
 {'Flour': 2, 'Butter': 7, 'Sugar': 7} 
 and fitness: 26320
0.001878976821899414
The individual's genome: 
 {'Flour': 2, 'Butter': 8, 'Sugar': 7} 
 and fitness: 27760
0.0021657943725585938
The individual's genome: 
 {'Flour': 4, 'Butter': 11, 'Sugar': 3} 
 and fitness: 28080
0.006744861602783203
The individual's genome: 
 {'Flour': 0, 'Butter': 6, 'Sugar': 11} 
 and fitness: 28880
0.0076410770416259766
The individual's genome: 
 {'Flour': 2, 'Butter': 11, 'Sugar': 6} 
 and fitness: 30240
0.0077168941497802734
The 